<a href="https://colab.research.google.com/github/flaviohds/analise_superstore/blob/main/superstore_ML_PQscript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
################################################################################
#
#   Script em python para previsao de vendas diarias utilizando um conselho de
# modelos de Machine Learning.
#   Mais informacoes em: https://github.com/flaviohds/analise_superstore
#
################################################################################

# Escolha a ultima data a ser prevista pelo modelo (YYYY/MM/DD)
last_date = '2019/12/31'

import pandas as pd
from numpy import pi, cos, sin, nan
from joblib import load
from urllib.request import urlopen
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.pipeline import Pipeline


# # Bloco de teste. Comente ou remova este bloco no script do PowerQuery ########
# # Importa o dataset do github
# url = 'https://raw.githubusercontent.com/flaviohds/analise_superstore/refs/heads/main/superstore_sales.csv'
# dataset = pd.read_csv(url, index_col=0)
# dataset = dataset.rename(columns={'Order Date': 'Data do pedido', 'Sales': 'Venda'})
# dataset.head()
# # Fim do bloco de teste #######################################################


# Converte a coluna de data
dataset['Data do pedido'] = pd.to_datetime(dataset['Data do pedido'], dayfirst=True)

# Agrupa a contagem de produtos vendidos e a soma dos valores por data
date_sum = dataset.groupby('Data do pedido')['Venda'].sum()

# Cria um dataframe com os agrupamentos
df_by_date = pd.DataFrame({'soma das vendas':date_sum.T})

# Deleta as variaveis que nao serao mais usadas
del dataset
del date_sum

# Adiciona a coluna para controle de venda prevista ou observada
df_by_date['predicted'] = 0

# Adiciona os dias faltantes que nao tiveram vendas
df_by_date = df_by_date.asfreq('D', fill_value=0)

# Cria 7 colunas de lag
for t in range(1,8):
  df_by_date['vendas_t-' + str(t)] = df_by_date['soma das vendas'].shift(t)

# Remove as primeiras linhas (que nao tinham os atributos de lag)
df_by_date.dropna(axis='index', how='any', inplace=True)

# Adiciona a ultima data a ser prevista e preenche o dataset com as datas
df_by_date.loc[pd.to_datetime(last_date)] = nan
df_by_date = df_by_date.asfreq('D')

# Copia a data que ficou no indice para uma nova coluna
df_by_date.insert(2, 'Date', df_by_date.index)

# Cria as colunas de data e descarta a coluna desnecessaria
df_by_date['Weekday'] = df_by_date['Date'].dt.weekday
df_by_date.insert(3, 'day_of_week_cos', cos(df_by_date['Weekday'] * (2*pi/7)))
df_by_date.insert(3, 'day_of_week_sin', sin(df_by_date['Weekday'] * (2*pi/7)))
df_by_date.insert(3, 'Day', df_by_date['Date'].dt.day)
df_by_date.insert(3, 'Year', df_by_date['Date'].dt.year)
df_by_date.insert(3, 'Month', df_by_date['Date'].dt.month)
df_by_date.drop(columns='Weekday', inplace=True)

# Reseta o indice
df_by_date = df_by_date.reset_index(drop=True)

# Carregando o modelo do github
url = 'https://raw.githubusercontent.com/flaviohds/analise_superstore/main/'
ml_model = load(urlopen(url+'superstore_model.pkl'))

# Loop para preencher os lags e prever as vendas. Itera apenas nas linhas novas
for index in df_by_date[df_by_date['predicted'] != 0].index:
  df_by_date.loc[index, 'predicted'] = 1

  df_by_date.loc[index, 'vendas_t-1'] = df_by_date.loc[index-1, 'soma das vendas']
  for t in range(2,8):
    df_by_date.loc[index, 'vendas_t-' + str(t)] = df_by_date.loc[index-1, 'vendas_t-' + str(t-1)]

  df_by_date.loc[index, 'soma das vendas'] = ml_model.predict(df_by_date.loc[index, 'Month':].values.reshape(1, -1))

# Remove as linhas observadas, colunas desnecessarias e modifica os '.' para ','
df_by_date = df_by_date[df_by_date['predicted'] != 0]
df_by_date = df_by_date[['soma das vendas', 'Date']]
df_by_date['soma das vendas'] = df_by_date['soma das vendas'].astype(str).str.replace('.', ',')